## SATELLITE TLE DATA LOADING

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import configparser
import time
from datetime import datetime

uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query/class/tle/EPOCH/2000-01-01--2020-01-31/NORAD_CAT_ID/24812, 26038, 27445, 27715, 27854, 27954, 28702, 28790, 28884, 28885, 29236, 31307, 32951, 33376, 33414, 36828, 37256, 37948, 38093, 40887, 39206, 40374, 37384, 40938, 32708, 19548, 41434/orderby/TLE_LINE1"
#requestCmdAction       = "/basicspacedata/query/class/tle/EPOCH/2015-01-01--2015-12-31/NORAD_CAT_ID/37158,%42738,%42965,%41622/orderby/TLE_LINE1"


# Use configparser package to pull in the ini file (pip install configparser)
config = configparser.ConfigParser()
config.read("./SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
configOut = config.get("configuration","output")
siteCred = {'identity': configUsr, 'password': configPwd}

print("URL::", uriBase + requestCmdAction)
# use requests package to drive the RESTful session with space-track.org
with requests.Session() as session:
    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    resp = session.get(uriBase + requestCmdAction)
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for satellites")

    # use the json package to break the json formatted response text into a Python structure (a list of dictionaries)
    retData = json.loads(resp.text)
    datFrame = pd.DataFrame(retData)
    datFrame = datFrame.reindex(sorted(datFrame.columns), axis=1)

    session.close()

# Parameters to derive apoapsis and periapsis from mean motion (see https://en.wikipedia.org/wiki/Mean_motion)
GM = 398600441800000.0
GM13 = GM ** (1.0/3.0)
MRAD = 6378.137
PI = 3.14159265358979
TPI86 = 2.0 * PI / 86400.0

def calcAPO(ecc, mmoti):
    sma = GM13 / ((TPI86 * mmoti) ** (2.0 / 3.0)) / 1000.0
    return sma * (1.0 + ecc) - MRAD

def calcPER(ecc, mmoti):
    sma = GM13 / ((TPI86 * mmoti) ** (2.0 / 3.0)) / 1000.0
    return sma * (1.0 - ecc) - MRAD

def calcOrbT(mmoti):
    sma = GM13 / ((TPI86 * mmoti) ** (2.0 / 3.0)) / 1000.0
    smak = sma * 1000.0
    return 2.0 * PI * ((smak ** 3.0) / GM) ** (0.5)

def calcOrbV(mmoti):
    sma = GM13 / ((TPI86 * mmoti) ** (2.0 / 3.0)) / 1000.0
    smak = sma * 1000.0
    return (GM / smak) ** (0.5)

#datFrame['APO'] = datFrame.apply(lambda row: calcAPO(float(row.ECCENTRICITY), float(row.MEAN_MOTION)), axis = 1)
#datFrame['PER'] = datFrame.apply(lambda row: calcPER(float(row.ECCENTRICITY), float(row.MEAN_MOTION)), axis = 1)
datFrame['OrbT'] = datFrame.apply(lambda row: calcOrbT(float(row.MEAN_MOTION)), axis = 1)
datFrame['OrbV'] = datFrame.apply(lambda row: calcOrbV(float(row.MEAN_MOTION)), axis = 1)

URL:: https://www.space-track.org/basicspacedata/query/class/tle/EPOCH/2000-01-01--2020-01-31/NORAD_CAT_ID/24812, 26038, 27445, 27715, 27854, 27954, 28702, 28790, 28884, 28885, 29236, 31307, 32951, 33376, 33414, 36828, 37256, 37948, 38093, 40887, 39206, 40374, 37384, 40938, 32708, 19548, 41434/orderby/TLE_LINE1


In [3]:
datFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118059 entries, 0 to 118058
Data columns (total 34 columns):
APOGEE                 118059 non-null object
ARG_OF_PERICENTER      118059 non-null object
BSTAR                  118059 non-null object
CLASSIFICATION_TYPE    118059 non-null object
COMMENT                118059 non-null object
DECAYED                118059 non-null object
ECCENTRICITY           118059 non-null object
ELEMENT_SET_NO         118059 non-null object
EPHEMERIS_TYPE         118059 non-null object
EPOCH                  118059 non-null object
EPOCH_MICROSECONDS     118059 non-null object
FILE                   118059 non-null object
INCLINATION            118059 non-null object
INTLDES                118059 non-null object
MEAN_ANOMALY           118059 non-null object
MEAN_MOTION            118059 non-null object
MEAN_MOTION_DDOT       118059 non-null object
MEAN_MOTION_DOT        118059 non-null object
NORAD_CAT_ID           118059 non-null object
OBJECT_ID      

In [115]:
datFrame.TLE_LINE0.unique()

array(['0 TDRS 3', '0 GALAXY 25 (TELSTAR 5)', '0 GALAXY 11',
       '0 GALAXY 3C', '0 GALAXY 12', '0 GALAXY 23 (TELSTAR 13)',
       '0 HORIZONS 1 (GALAXY 13)', '0 GALAXY 28 (IA 8)', '0 GALAXY 14',
       '0 GALAXY 15', '0 SYRACUSE 3A', '0 GALAXY 16', '0 GALAXY 17',
       '0 AMC-14', '0 GALAXY 18', '0 GALAXY 19', '0 VENESAT-1',
       '0 BEIDOU 5', '0 BEIDOU 7', '0 BEIDOU 8', '0 BEIDOU 10',
       '0 MUOS 1', '0 MUOS 2', '0 MUOS 3', '0 MUOS 4', '0 BD-20',
       '0 BEIDOU IGSO-6'], dtype=object)

In [116]:
datFrame.to_csv('NewSatDataExtract.csv')

In [117]:
SatDataExt = pd.read_csv('NewSatDataExtract.csv', engine='python')
SatDataExtDF = pd.DataFrame(SatDataExt, columns= SatDataExt.columns)
SatDataExtDF = SatDataExtDF.loc[:, ~SatDataExtDF.columns.str.contains('^Unnamed')]
SatDataExtDF.info()

,APOGEE,ARG_OF_PERICENTER,BSTAR,CLASSIFICATION_TYPE,COMMENT,DECAYED,ECCENTRICITY,ELEMENT_SET_NO,EPHEMERIS_TYPE,EPOCH,...,PERIGEE,PERIOD,RA_OF_ASC_NODE,REV_AT_EPOCH,SEMIMAJOR_AXIS,TLE_LINE0,TLE_LINE1,TLE_LINE2,OrbT,OrbV
0,35801.617,194.5447,0.0001,U,GENERATED VIA SPACETRACK.ORG API,0,0.000331,999,0,2000-01-03 18:38:31,...,35773.720,1436.150,65.0207,2852,42165.804,0 TDRS 3,1 19548U 88091B 00003.77675002 -.00000187 0...,2 19548 004.9738 065.0207 0003308 194.5447 208...,86169.018775,3074.601482
1,35801.476,199.1097,0.0001,U,GENERATED VIA SPACETRACK.ORG API,0,0.000327,999,0,2000-01-03 22:14:42,...,35773.900,1436.151,65.0208,2852,42165.823,0 TDRS 3,1 19548U 88091B 00003.92688147 -.00000187 0...,2 19548 004.9745 065.0208 0003270 199.1097 257...,86169.078073,3074.600777
2,35801.498,200.1807,0.0001,U,GENERATED VIA SPACETRACK.ORG API,0,0.000326,999,0,2000-01-04 22:23:56,...,35773.989,1436.154,65.0156,2853,42165.879,0 TDRS 3,1 19548U 88091B 00004.93329369 -.00000186 +0...,2 19548 004.9763 065.0156 0003262 200.1807 259...,86169.249091,3074.598743
3,35802.474,213.1003,0.0001,U,GENERATED VIA SPACETRACK.ORG API,0,0.000344,999,0,2000-01-08 18:59:55,...,35773.438,1436.164,64.9980,2857,42166.091,0 TDRS 3,1 19548U 88091B 00008.79161543 -.00000183 +0...,2 19548 004.9838 064.9980 0003443 213.1003 199...,86169.899655,3074.591005
4,35802.684,213.5764,0.0001,U,GENERATED VIA SPACETRACK.ORG API,0,0.000348,999,0,2000-01-09 16:42:30,...,35773.320,1436.167,64.9952,2858,42166.137,0 TDRS 3,1 19548U 88091B 00009.69618317 -.00000183 +0...,2 19548 004.9855 064.9952 0003482 213.5764 165...,86170.039738,3074.589339


## SOLAR WAVE DATA LOADING

In [101]:
from pandas import read_excel
my_sheet = 'Sheet1'
file_name = 'SWDataExtractAll.xlsx'
SolarWaveDataDF = read_excel(file_name, sheet_name = my_sheet)

In [102]:
SolarWaveDataDF.head(5)

,yyyy,mm,dd,BSRN,ND,Kp,Kp.1,Kp.2,Kp.3,Kp.4,...,Cp,C9,ISN,Adj F10.7,Q,Adj Ctr81,Adj Lst81,Obs F10.7,Obs Ctr81,Obs Lst81
0,1957,10,1,1700,19,43,40,30,20,37,...,1.1,5.0,236.0,268.0,0.0,265.2,230.6,269.3,266.6,230.9
1,1957,10,2,1700,20,37,37,17,17,27,...,0.7,3.0,234.0,252.0,0.0,266.0,231.4,253.3,267.4,231.7
2,1957,10,3,1700,21,27,20,13,33,37,...,1,5.0,242.0,265.0,0.0,266.7,232.3,266.3,268.1,232.7
3,1957,10,4,1700,22,30,30,23,27,23,...,0.7,3.0,217.0,237.0,0.0,267.4,232.9,238.2,268.8,233.3
4,1957,10,5,1700,23,30,30,17,23,20,...,0.6,3.0,219.0,245.0,0.0,267.8,233.5,246.2,269.3,233.9


In [103]:
SolarWaveDataDF['Date'] = SolarWaveDataDF[SolarWaveDataDF.columns[0:3]].apply(
    lambda x: '-'.join(x.dropna().astype(str)),
    axis=1)

In [105]:
SolarWaveDataDF = SolarWaveDataDF.drop(['yyyy','mm','dd'], axis = 1)
SolarWaveDataDF.head(5)

,BSRN,ND,Kp,Kp.1,Kp.2,Kp.3,Kp.4,Kp.5,Kp.6,Kp.7,...,C9,ISN,Adj F10.7,Q,Adj Ctr81,Adj Lst81,Obs F10.7,Obs Ctr81,Obs Lst81,Date
0,1700,19,43,40,30,20,37,23.0,43,37,...,5.0,236.0,268.0,0.0,265.2,230.6,269.3,266.6,230.9,1957-10-1
1,1700,20,37,37,17,17,27,23.0,17,30,...,3.0,234.0,252.0,0.0,266.0,231.4,253.3,267.4,231.7,1957-10-2
2,1700,21,27,20,13,33,37,47.0,43,30,...,5.0,242.0,265.0,0.0,266.7,232.3,266.3,268.1,232.7,1957-10-3
3,1700,22,30,30,23,27,23,27.0,30,27,...,3.0,217.0,237.0,0.0,267.4,232.9,238.2,268.8,233.3,1957-10-4
4,1700,23,30,30,17,23,20,27.0,27,20,...,3.0,219.0,245.0,0.0,267.8,233.5,246.2,269.3,233.9,1957-10-5


In [106]:
SolarWaveDataDF = SolarWaveDataDF.rename(columns={"Sum": "Kp Sum", "Avg": "Ap Avg", "ISN": "SSN"})

In [107]:
SolarWaveDataDF.describe()

,Kp.5,Kp Sum,Ap.2,Ap.5,Ap Avg,C9,SSN,Adj F10.7,Q,Adj Ctr81,Adj Lst81,Obs F10.7,Obs Ctr81,Obs Lst81
count,22819.000000,22819.000000,22819.000000,22819.000000,22818.000000,22707.000000,22773.000000,22948.000000,22773.000000,22948.000000,22948.000000,22948.000000,22948.000000,22948.000000
mean,21.373242,172.455629,12.437721,12.949253,13.078754,2.568943,65.256795,121.462084,0.075308,121.445041,121.744182,121.612685,121.593751,121.892191
std,14.295898,94.865163,18.623980,19.125973,14.890422,2.130930,65.211238,51.689610,0.317105,47.783552,48.016828,51.978584,48.072026,48.308050
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,100.000000,4.000000,4.000000,5.000000,1.000000,16.000000,77.100000,0.000000,78.800000,78.900000,77.400000,78.700000,78.800000
50%,20.000000,160.000000,7.000000,7.000000,9.000000,2.000000,51.000000,105.300000,0.000000,108.600000,108.900000,105.300000,108.500000,108.700000
75%,30.000000,233.000000,15.000000,15.000000,16.000000,4.000000,100.000000,152.725000,0.000000,154.900000,155.300000,152.800000,155.500000,156.000000
max,90.000000,667.000000,400.000000,400.000000,280.000000,9.000000,4605.000000,377.000000,4.000000,276.000000,276.000000,383.400000,279.600000,279.600000


In [108]:
SolarWaveDataDF = SolarWaveDataDF.rename(columns={"Kp": "Kp 0-3 Hrs", "Kp.1": "Kp 3-6 Hrs",
                                                 "Kp.2": "Kp 6-9 Hrs", "Kp.3": "Kp 9-12 Hrs",
                                                 "Kp.4": "Kp 12-15 Hrs", "Kp.5": "Kp 15-18 Hrs",
                                                 "Kp.6": "Kp 18-21 Hrs", "Kp.7": "Kp 21-24 Hrs"})

In [109]:
SolarWaveDataDF = SolarWaveDataDF.rename(columns={"Ap": "Ap 0-3 Hrs", "Ap.1": "Ap 3-6 Hrs",
                                                 "Ap.2": "Ap 6-9 Hrs", "Ap.3": "Ap 9-12 Hrs",
                                                 "Ap.4": "Ap 12-15 Hrs", "Ap.5": "Ap 15-18 Hrs",
                                                 "Ap.6": "Ap 18-21 Hrs", "Ap.7": "Ap 21-24 Hrs"})

In [110]:
SolarWaveDataDF.head(5)

,BSRN,ND,Kp 0-3 Hrs,Kp 3-6 Hrs,Kp 6-9 Hrs,Kp 9-12 Hrs,Kp 12-15 Hrs,Kp 15-18 Hrs,Kp 18-21 Hrs,Kp 21-24 Hrs,...,C9,SSN,Adj F10.7,Q,Adj Ctr81,Adj Lst81,Obs F10.7,Obs Ctr81,Obs Lst81,Date
0,1700,19,43,40,30,20,37,23.0,43,37,...,5.0,236.0,268.0,0.0,265.2,230.6,269.3,266.6,230.9,1957-10-1
1,1700,20,37,37,17,17,27,23.0,17,30,...,3.0,234.0,252.0,0.0,266.0,231.4,253.3,267.4,231.7,1957-10-2
2,1700,21,27,20,13,33,37,47.0,43,30,...,5.0,242.0,265.0,0.0,266.7,232.3,266.3,268.1,232.7,1957-10-3
3,1700,22,30,30,23,27,23,27.0,30,27,...,3.0,217.0,237.0,0.0,267.4,232.9,238.2,268.8,233.3,1957-10-4
4,1700,23,30,30,17,23,20,27.0,27,20,...,3.0,219.0,245.0,0.0,267.8,233.5,246.2,269.3,233.9,1957-10-5


In [111]:
SolarWaveDataDF.to_csv('SWDataExtAll.csv')

In [112]:
SolarWaveDataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22962 entries, 0 to 22961
Data columns (total 31 columns):
BSRN            22959 non-null object
ND              22957 non-null object
Kp 0-3 Hrs      22826 non-null object
Kp 3-6 Hrs      22823 non-null object
Kp 6-9 Hrs      22822 non-null object
Kp 9-12 Hrs     22821 non-null object
Kp 12-15 Hrs    22820 non-null object
Kp 15-18 Hrs    22819 non-null float64
Kp 18-21 Hrs    22819 non-null object
Kp 21-24 Hrs    22819 non-null object
Kp Sum          22819 non-null float64
Ap 0-3 Hrs      22819 non-null object
Ap 3-6 Hrs      22819 non-null object
Ap 6-9 Hrs      22819 non-null float64
Ap 9-12 Hrs     22819 non-null object
Ap 12-15 Hrs    22819 non-null object
Ap 15-18 Hrs    22819 non-null float64
Ap 18-21 Hrs    22819 non-null object
Ap 21-24 Hrs    22819 non-null object
Ap Avg          22818 non-null float64
Cp              22707 non-null object
C9              22707 non-null float64
SSN             22773 non-null float64
Adj F10.7 

#### Solar Wave Data Description

#### Kp-index
The K-index, and by extension the Planetary K-index, are used to characterize the magnitude of geomagnetic storms. Kp is an excellent indicator of disturbances in the Earth's magnetic field and is used by SWPC to decide whether geomagnetic alerts and warnings need to be issued for users who are affected by these disturbances.

The principal users affected by geomagnetic storms are the electrical power grid, spacecraft operations, users of radio signals that reflect off of or pass through the ionosphere, and observers of the aurora.

The Estimated 3-hour Planetary Kp-index is derived at the NOAA Space Weather Prediction Center using data from the following ground-based magnetometers: Sitka, Alaska; Meanook, Canada; Ottawa, Canada; Fredericksburg, Virginia; Hartland, UK; Wingst, Germany; Niemegk, Germany; and Canberra, Australia. These data are made available thanks to the cooperative efforts between SWPC and data providers around the world, which currently includes the U.S. Geological Survey, Natural Resources Canada (NRCAN), the British Geological Survey, the German Research Centre for Geosciences (GFZ), and Geoscience Australia. Important magnetometer observations are also contributed by the Institut de Physique du Globe de Paris and the Korean Space Weather Center K-index Watches are issued when the highest predicted NOAA estimated Kp-indices for a day are K = 5, 6, 7, or >= 8 and is reported in terms of the NOAA G scale. K-index Warnings are issued when NOAA estimated Kp-indices of 4, 5, 6, and 7 or greater are expected. K-index Alerts are issued when the NOAA estimated Kp-indices reach 4, 5, 6, 7, 8, or 9. 

#### Ap Index
The Ap-index provides a daily average level for geomagnetic activity. Because of the non-linear relationship of the K-scale to magnetometer fluctuations, it is not meaningful to take the average of a set of K-indices. Instead, every 3-hour K-value will be converted back into a linear scale called the a-index. The average from 8 daily a-values gives us the Ap-index of a certain day. The Ap-index is thus a geomagnetic activity index where days with high levels of geomagnetic activity have a higher daily Ap-value.


#### BSRN
The Baseline Surface Radiation Network (BSRN) was implemented by the World Climate Research Programme (WCRP) starting observations with nine stations in 1992, under the auspices of the World Meteorological Organization (WMO). Currently, 59 BSRN stations submit their data to the WCRP.
The basic-BSRN measurements of the BSRN program at IZA are global shortwave radiation (SWD), direct radiation (DIR), diffuse radiation (DIF) and longwave downward radiation (LWD)
SWD and DIF are measured with unshaded and shaded EKO MS-802F pyranometers (Fig. 2a and b) (ISO-9060 classification: secondary standard), respectively. This pyranometer is a high-precision instrument with a spectral range from 285 to 3000 nm with a response time less than 5 s (95 %, confidence level) and an expected uncertainty <±1 % for daily totals. The BSRN accuracy target for DIF and SWD is 2 % (5 W m−2) and 2 % (3 W m−2), respectively.

#### ISN
ISN is also knows as SSN -Smoothed sunspot number (SSN)

Space Weather Indices

In any science, if we are going to progress, we need to make quantitative measurements. Scientists sometimes refer to these as meter readings, since many sensors used to measure quantities have a meter to display the quantity of interest. Averages of these readings in a particular environment may be referred to as indices. They are then used to describe the state of that environment.

In terrestrial meteorology, there are several environmental parameters of which we are aware. These include temperature, pressure, humidity, wind speed and direction, and precipitation. These values change with time and with place. Averages of these values help us judge climate trends and overall conditions. In space weather there are also indices which help us to judge and monitor the state of the space environment.

Space weather indices include sunspot number, geomagnetic indices, solar wind parameters (density and speed), flare index, solar x-ray flux and many more. As our knowledge of space weather progresses new indices will undoubtedly arise and old indices will be consolidated.

Two of the most used space weather indices are smoothed sunspot number (SSN) and the geomagnetic planetary A index (Ap). You may also hear mention of the solar ten centimetre radio flux (F10.7). This is closely related to SSN. Both SSN and F10.7 give an indication of the overall level of solar activity. SSN ranges from zero to over 300. Although this value is said to indicate solar activity, it does not always mean activity with regard to flares and coronal mass ejections. It might be regarded as similar to a space weather temperature, but we must be careful with such analogies.

The table below indicates the level of solar activity as the sunspot number changes.

SSN	Solar Activity
>250	Extreme,
150-250	Very High,
80-150	High,
40-80	Moderate,
20-40	Low,
0-20	Very Low
The Ap index, and its logarithmic cousin Kp, give a measure of the storminess of the Earth's magnetic field. Ap may range from 0 to about 400. The table below indicates the level of geomagnetic activity as the value of Ap changes.

Ap	Geomagnetic Activity
>100	Severe Storm,
50-99	Major Storm,
30-49	Minor Storm,
16-29	Active,
8-15	Unsettled,
0-7	Quiet

The following conversions between Ap and Kp and between SSN and F10.7 are sometimes useful:

Ap	0	4	7	15	27	48	80	132	207	400
Kp	0	1	2	3	4	5	6	7	8	9

SSN	0	25	50	75	100	125	150	175	200	250
F10.7	67	83	102	124	148	172	196	219	240	273
